#### Load all dependencies

In [1]:
import numpy as np
import pandas as pd 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
from tqdm import tqdm    # for progress bar
import os

#### Set device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### Define Transform

In [3]:
train_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')), # ensures RGB 
    transforms.Resize((224,224)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

test_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

#### Load data

In [4]:
train_data = ImageFolder('/kaggle/input/cat-dot-pandas-dataset/Cat-Dog_Pandas/Train',
                        transform = train_transform)
val_data = ImageFolder('/kaggle/input/cat-dot-pandas-dataset/Cat-Dog_Pandas/Valid',
                      transform = test_transform)

In [5]:
class_names = train_data.classes   # different classes in our datasets
class_names

['cat', 'dog', 'panda']

#### Data loader

In [6]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [7]:
dataiter = iter(train_loader)
features,label = next(dataiter)
print('number of data per batch: ',len(features))
print('number of label per batch: ',len(label))
print('labels : ',label)

number of data per batch:  32
number of label per batch:  32
labels :  tensor([2, 0, 1, 2, 1, 2, 0, 2, 2, 1, 2, 1, 2, 0, 1, 0, 2, 0, 2, 1, 2, 2, 1, 0,
        2, 0, 0, 0, 1, 0, 0, 2])


#### ResNet50 Tensfor Learning technique

In [8]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # 3 classes dog, cat, pandas
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 206MB/s]


#### Loss and Optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Training Loop

In [10]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Wrap train_loader with tqdm
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")
    
    for images, labels in train_loader_tqdm:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Update tqdm bar postfix with running metrics
        train_loader_tqdm.set_postfix({
            'loss': f"{loss.item():.4f}"
        })

    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation")
    
    with torch.no_grad():
        for images, labels in val_loader_tqdm:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            val_loader_tqdm.set_postfix({
                'loss': f"{loss.item():.4f}"
            })

    val_loss /= total_val
    val_acc = correct_val / total_val

    # Final log for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch [1/15] Validation: 100%|██████████| 10/10 [00:03<00:00,  3.33it/s, loss=0.0047]


Epoch [1/15] Train Loss: 0.6632, Train Acc: 0.7429 Val Loss: 0.5666, Val Acc: 0.8067


Epoch [2/15] Validation: 100%|██████████| 10/10 [00:01<00:00,  5.54it/s, loss=0.1739]


Epoch [2/15] Train Loss: 0.4220, Train Acc: 0.8362 Val Loss: 0.4145, Val Acc: 0.8233


Epoch [3/15] Validation: 100%|██████████| 10/10 [00:01<00:00,  5.58it/s, loss=0.3323]


...

Epoch [13/15] Train Loss: 0.0713, Train Acc: 0.9762 Val Loss: 0.3322, Val Acc: 0.9133


Epoch [14/15] Validation: 100%|██████████| 10/10 [00:01<00:00,  5.53it/s, loss=0.0005]


Epoch [14/15] Train Loss: 0.0429, Train Acc: 0.9876 Val Loss: 0.2686, Val Acc: 0.9167


Epoch [15/15] Validation: 100%|██████████| 10/10 [00:01<00:00,  5.54it/s, loss=0.0000]

Epoch [15/15] Train Loss: 0.0336, Train Acc: 0.9905 Val Loss: 0.6272, Val Acc: 0.8800


#### Model testing with unseen images

In [11]:
def RandomImagePrediction(filepath):
    img_array = Image.open(filepath).convert("RGB")
    data_transforms=transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(), 
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    img = data_transforms(img_array).unsqueeze(dim=0) # Returns a new tensor with a dimension of size one inserted at the specified position.
    load = DataLoader(img)
    
    for x in load:
        x=x.to(device)
        pred = model(x)
        _, preds = torch.max(pred, 1)
        print(f"class : {preds}")

        
        if preds[0] == 0: print(f"predicted ----> Cat")
        
        elif preds[0] == 1: print(f"predicted ----> Dog")

        else : print(f'predicted ---> Pandas')

In [12]:
if __name__ == "__main__":
    RandomImagePrediction("/kaggle/input/cat-dot-pandas-dataset/Cat-Dog_Pandas/Test/104000.jpg") # cat image
    RandomImagePrediction("/kaggle/input/cat-dot-pandas-dataset/Cat-Dog_Pandas/Test/133600.jpg") # dog image
    RandomImagePrediction("/kaggle/input/cat-dot-pandas-dataset/Cat-Dog_Pandas/Test/149900.jpg") # panda image

class : tensor([0], device='cuda:0')
predicted ----> Cat
class : tensor([1], device='cuda:0')
predicted ----> Dog
class : tensor([2], device='cuda:0')
predicted ---> Pandas


#### Save Trained Model

In [13]:
torch.save(model.state_dict(), "resnet50_cat_dog_panda.pth")
print("Model saved as resnet50_cat_dog_panda.pth")

Model saved as resnet50_cat_dog_panda.pth
